<a href="https://colab.research.google.com/github/Shakorly/A-B-Test/blob/main/ronaldo_hazard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/2_way_image_classification/image_modelling/test_data',  # Update with your dataset path
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/2_way_image_classification/image_modelling/validate_data',  # Update with your dataset path
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load pre-trained model and add custom layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)  # We have 2 classes: Ronaldo and Hazard

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Save the model
model.save('player_classification_model.h5')


Found 223 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/10
7/7 [==============================] - 81s 11s/step - loss: 1.3919 - accuracy: 0.4664 - val_loss: 1.0672 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 60s 9s/step - loss: 0.8158 - accuracy: 0.5605 - val_loss: 0.8104 - val_accuracy: 0.5000
Epoch 3/10
7/7 [==============================] - 61s 9s/step - loss: 0.7504 - accuracy: 0.5471 - val_loss: 0.8305 - val_accuracy: 0.5000
Epoch 4/10
7/7 [==============================] - 68s 10s/step - loss: 0.7049 - accuracy: 0.5247 - val_loss: 0.7062 - val_accuracy: 0.5667
Epoch 5/10
7/7 [==============================] - 61s 9s/step - loss: 0.6481 - accuracy: 0.6099 - val_loss: 0.7084 - val_accuracy: 0.6000
Epoch 6/10
7/7 [==============================] - 68s 10s/step - loss: 0.6175 - accuracy: 0.6547 - val_loss: 0.7202 - val_accuracy: 0.6000
Epoch 7/10
7/7 [=======

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

# Load the data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/2_way_image_classification/image_modelling/test_data',  # Update with your dataset path
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/2_way_image_classification/image_modelling/validate_data',  # Update with your dataset path
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Make the top layers trainable
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 20
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs = 20,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

# Save the model
model.save('player_classification_model.h5')


Found 223 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/20
7/7 [==============================] - 83s 10s/step - loss: 0.7660 - accuracy: 0.5291 - val_loss: 0.6817 - val_accuracy: 0.5000
Epoch 2/20
7/7 [==============================] - 7s 1s/step - loss: 0.5994 - accuracy: 0.7085 - val_loss: 0.6755 - val_accuracy: 0.5333
Epoch 3/20
7/7 [==============================] - 6s 875ms/step - loss: 0.5146 - accuracy: 0.7623 - val_loss: 0.7386 - val_accuracy: 0.5000
Epoch 4/20
7/7 [==============================] - 6s 884ms/step - loss: 0.5258 - accuracy: 0.7354 - val_loss: 0.7360 - val_accuracy: 0.5000
Epoch 5/20
7/7 [==============================] - 7s 1s/step - loss: 0.4048 - accuracy: 0.8072 - val_loss: 0.7590 - val_accuracy: 0.5000
Epoch 6/20
7/7 [==============================] - 5s 764ms/step - loss: 0.4253 - accuracy: 0.8072 - val_loss: 0.7670 - val_accuracy: 0.5000
Epoch 7/20
7/7 [=====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
